In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>")) 

In [2]:
%matplotlib inline
import os
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from obspy import Trace, Stream, UTCDateTime
from matplotlib.mlab import psd, magnitude_spectrum
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage import feature
import matplotlib.image as mpimg
import matplotlib.dates as mdates

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.ticker import FormatStrFormatter
from matplotlib.gridspec import GridSpec

from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
from skimage.morphology import skeletonize
from skimage.transform import rotate, hough_line, hough_line_peaks
import scipy.signal as ss

from obspy.signal.spectral_estimation import get_nlnm, get_nhnm

import matplotlib as mpl
mpl.rcParams['date.autoformatter.day'] = "%Y-%m-%d"
mpl.rcParams['date.autoformatter.hour'] = "%Y-%m-%d %Hh"
mpl.rcParams['date.autoformatter.minute'] = "%Y-%m-%d %H:%M"

mpl.rcParams['figure.figsize'] = "12,8"
mpl.rcParams['figure.dpi'] = 100

mpl.rcParams['axes.axisbelow'] = True


In [3]:
import seaborn as sns

In [4]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])


In [16]:
traces = {}

In [17]:
from PIL import Image
import PIL.ImageOps  

files = sorted(glob.glob(r"data\19530*.jpg"))

PLOT = False
# Galitzin: bottom to up, left to right

for file in files:
    print("Processing %s"%file)
    filename = os.path.split(file)[1]
    if filename in traces:
        continue

    im = Image.open(file)
#     im = im.rotate(90)
    im = PIL.ImageOps.invert(im)

    im = np.asarray(im).copy()
    print(im.shape)
    if len(im.shape) > 2:
        im = rgb2gray(im) 

    if PLOT:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.imshow(im, aspect='auto', interpolation="none")
        plt.savefig("image_0_raw.pdf")
    # apply threshold
    image = im.copy()
    thresh = threshold_otsu(image)

#     if PLOT:
#         fig, ax = plt.subplots(figsize=(10, 6))
#         ax.imshow(thresh, aspect='auto', interpolation="none")
#         plt.savefig("image_1_threshold.png")
    
    bw = closing(image > thresh, square(3))
    
    if PLOT:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.imshow(bw, aspect='auto', interpolation="none")
        plt.savefig("image_2_bw.pdf")
    
    
    # remove artifacts connected to image border
    cleared = clear_border(bw)
    
    if PLOT:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.imshow(cleared, aspect='auto', interpolation="none")
        plt.savefig("image_3_cleared.pdf")
    
    
    # find best rotation to make sure traces are elongated horizontally
#     if PLOT: plt.figure()
#     angles = np.arange(-2,2,0.1)
#     rotations = []
#     for angle in angles:
#         tmp = rotate(cleared, float(angle))
#         if PLOT: plt.plot(tmp.sum(axis=1), label="angle=%f"%angle)
#         rotations.append(tmp.sum(axis=1).max())
#     print(np.max(rotations), angles[np.argmax(rotations)])
    
    angles = np.deg2rad(np.linspace(85,95,200))
    hspace, angles, dists = hough_line(cleared, theta=angles)
    hspace, angles, dists = hough_line_peaks(hspace, angles, dists)
    angle = np.median(np.rad2deg(angles)) - 90
    
#     if PLOT: 
#         plt.legend()
#         plt.show()
    
    cleared = rotate(cleared, angle)
    
    if PLOT:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.imshow(cleared, aspect='auto', interpolation="none")
        plt.savefig("image_4_rotated.pdf")
    
    
    print(cleared.shape)
    # label image regions
    label_image = label(cleared, connectivity=2)
    print(label_image.shape)
    image_label_overlay = label2rgb(label_image, image=cleared, bg_label=0, alpha=0.8)
    if PLOT:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.imshow(image_label_overlay, aspect='auto', interpolation="none")
        plt.savefig("image_5_label_overlay.pdf")
    i = 0
    all = []
    all_s = {}
    all_box = []
    all_id = {}
    for region in regionprops(label_image):
        # take regions with large enough area
        if region.area >= 100:
            # check zone size, if too small on time axis or too wide on amplitude, reject
            _minr, _minc, _maxr, _maxc = region.bbox
            if _maxc-_minc <= 50:
                continue
            if _maxr - _minr >= 100:
                continue
            # draw rectangle around segmented traces
            minr, minc, maxr, maxc = region.bbox
            if PLOT:
                rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                          fill=False, edgecolor='red', linewidth=0.1)
                plt.text((maxc+minc)/2, minr, "%05i" %i, color='w', fontsize=2, horizontalalignment="center")
                ax.add_patch(rect)
            i+=1
            tmp = cleared[minr:maxr,minc:maxc]
            tmp -= tmp.min()
            tmp[tmp>0] = 1.0
            skeleton = skeletonize(tmp)
            xx = []
            for _ in range(skeleton.shape[1]):
                _ = skeleton[:,_]
                __ = np.where(_==1)[0]
                if len(__):
                    xx.append(__[0])
                else:
                    if len(xx):
                        xx.append(xx[-1])
                    else:
                        xx.append(0)
            _ = np.asarray(xx, dtype='float')
            if _[0] == 0:
                _[0] = _[1]

            if PLOT:
                plt.plot(np.arange(minc,maxc),minr+_, c='w', lw=0.1, zorder=100)
            _ -= _.mean()
            trace = {"x": minc, "y":(minr+maxr)/2., "data":_ }
            all.append(trace)

    traces[filename] = all
    if PLOT:
        ax.set_axis_off()
        plt.tight_layout()
        plt.savefig("image_6_skeleton.pdf")
#         plt.show()
    del image, im, label_image, bw, cleared, image_label_overlay
    break
print("Done")
plt.show()

Processing data\19530128.jpg
(3576, 11109)
(3576, 11109)
(3576, 11109)
Done


In [18]:
traces.keys()

dict_keys(['19530128.jpg'])

In [19]:
start_times = {}
start_times["19530128.jpg"] = "07:45:00"
start_times["19530129.jpg"] = "07:43:00"


In [20]:
processed_traces = []


for key in sorted(traces):
    print(key)
    date,_ = key.split('.')
    d = datetime.datetime.strptime(date+ " %s" % start_times[key], "%Y%m%d %H:%M:%S")
    df = pd.DataFrame(traces[key].copy())
    df = df.sort_values(["y","x"], ascending=[True,True])
    ppmin = np.median([len(a) for a in df["data"]])
    print("ppmin", ppmin)
    pps = ppmin / 59.0
    timer = 0
    for id, row in df.iterrows():
        tr = Trace(data=row["data"].copy())
        tr.data *= 0.0254/300.0 # data is now in meters
        if id == 10:
            print(tr.data.ptp())
        tr.stats.sampling_rate = pps
        tr.stats.starttime = UTCDateTime(d) + timer
        tr.detrend("linear")
        tr.interpolate(8, method='lanczos', a=64)
        tr.taper(None, max_length=0.5, side="both")
        tr.filter("highpass", freq=0.05, corners=8)
        if id == 10:
            print(tr.data.ptp())
        processed_traces.append(tr)
        timer += tr.stats.npts*tr.stats.delta + 1


19530128.jpg
ppmin 352.0


C:\ProgramData\Anaconda3\envs\py37\lib\site-packages\obspy\core\trace.py:2111: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(self.data.dtype, float):


0.0014393333333333333
0.001588333226781628


In [21]:
st = Stream(traces=processed_traces)
st.write("1953.BHZ.mseed")
